# Лабораторная работа № 6. Эмбеддинги и обработка текстов
## Будем рассматривать задачу классификации текстов на 2 категории: тексты статей сайта https://meduza.io/ и https://habr.com/ru/.
Для работы с текстами нам понадобятся несколько дополнительных библиотек:
nltk - большая библиотека, в которой содержится множество полезных функций для обработки текста, такие, как, например, удаление пунктуации, стемминг и лемматизацию
snowballstemmer - стеммер для русского языка
pymorphy2, pymystem3 - морфоанализаторы для русского языка
gensim - библиотека, содержащая модели текстовых эмбеддингов

In [ ]:
!pip install nltk snowballstemmer pymorphy2 pymystem3 gensim

In [53]:
import nltk
import pymystem3
import pymorphy2
from snowballstemmer import RussianStemmer
nltk.download('punkt')
nltk.download('stopwords')
stemmer = RussianStemmer()
morph = pymorphy2.MorphAnalyzer()
mystem = pymystem3.Mystem()

from string import punctuation
from nltk.corpus import stopwords
from pymystem3 import Mystem


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dimak\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dimak\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Напишем функцию леммитизации наших данных для pandas.

In [54]:
from string import punctuation
#Create lemmatizer and stopwords list
mystem = Mystem() 
russian_stopwords = stopwords.words("russian")

#Preprocess function
def preprocess_text(text):
    tokens = mystem.lemmatize(text.lower())
    tokens = [token for token in tokens if token not in russian_stopwords\
              and token != " " \
              and token.strip() not in punctuation]
    
    text = " ".join(tokens)
    
    return text


## Подготовим Текст.
###  На данном этапе оставим только наши тексты исходя из правил:
1. 50 текстов каждого класса
2. В каждом тексте порядка 2к слов

Хотя в голове были мысли 

![title](mem/3.jpg)

In [55]:
import numpy as np
import pandas as pd


## Рассмотрим задачу бинарной классификации текстов. Возьмем набор текстов статей медузы и хабра  и попробуем подобрать такой алгоритм, который смог бы их отличать друг от друга.

In [56]:
import pandas as pd

data= pd.read_csv('meduza.csv', sep=',')

In [57]:
data['count'] = data['text'].str.count(' ') + 1
srt = data.sort_values(['count'], ascending=[False])
srt = srt[srt['count'].between(2000, 2070)]

srt.head()
meduza=srt.drop(['date', 'url', 'edition', 'topics', 'authors', 'title', 'reposts_fb', 'reposts_vk',
                 'reposts_ok', 'reposts_twi', 'reposts_lj', 'reposts_tg', 
                 'likes',  'views', 'comm_count', 'count' ], axis='columns')
meduza['class'] = 1

In [58]:
meduza.head()

,text,class
31845,23 апреля 2017 года на ютьюбе запустилось «Дру...,1
25101,"Есть неизлечимые заболевания, при которых дети...",1
24227,В начале декабря в издательстве «КоЛибри»nвыхо...,1
25130,Борис Березовский на организованном им пикете ...,1
62311,В Киеве в середине июня решилиnрасформироватьn...,1


In [59]:

#Create lemmatizer and stopwords list
mystem = Mystem() 
russian_stopwords = stopwords.words("russian")

#Preprocess function
def preprocess_text(text):
    tokens = mystem.lemmatize(text.lower())
    tokens = [token for token in tokens if token not in russian_stopwords\
              and token != " " \
              and token.strip() not in punctuation]
    
    text = " ".join(tokens)
    
    return text

In [60]:
meduza['text'] = meduza.apply(lambda row: preprocess_text(row['text']), axis=1)

In [61]:
meduza.head()

,text,class
31845,23 апрель 2017 год ютьюбе запуститься « дружк...,1
25101,неизлечимый заболевание который ребенок мочь д...,1
24227,начало декабрь издательство « колибри » пвыхо...,1
25130,борис березовский организованный пикет консуль...,1
62311,киев середина июнь решилипрасформироватьподин ...,1


In [62]:
patients_df = pd.read_json('texts.json' , lines=True)
patients_df = patients_df.replace(r'\n',' ', regex=True) 
patients_df.head()

,text,link
0,"Марк Шаттлворт объявил о том, что кодовым н...",https://habr.com/ru/post/129837/
1,Введение В марте 2015 года Epic Games поверну...,https://habr.com/ru/post/341756/
2,"Выступление COO FunCorp, Дениса Литвинова, на...",https://habr.com/ru/company/mobio/blog/341826/
3,Здравствуйте! Команда ONLYOFFICE возвращается...,https://habr.com/ru/company/teamlab/blog/341522/
4,По умолчанию в Parallels Desktop 13 выставл...,https://habr.com/ru/company/parallels/blog/341...


In [63]:
patients_df['count'] = patients_df['text'].str.count(' ') + 1
habr = patients_df.sort_values(['count'], ascending=[False])
habr = habr[habr['count'].between(2000, 2032)]
habr = habr.drop(['link', 'count'], axis='columns')
habr['class'] = 0


In [65]:
habr['text'] = habr.apply(lambda row: preprocess_text(row['text']), axis=1)

In [66]:

habr.head()

,text,class
6427,последний шесть год пока строить команда проек...,0
9743,наш время оптический волокно становиться неотъ...,0
6772,снова здравствовать большой маленький любитель...,0
4161,большинство человек привыкать chromium — это...,0
9088,"“ выживать сильный ,” — пафосный заявлять вс...",0


In [67]:
df = pd.DataFrame(columns=['text', 'class'])
df = df.append(habr, ignore_index=True)

### Посмотрим на вид нашего датафрейма

In [68]:
df.head()

,text,class
0,последний шесть год пока строить команда проек...,0
1,наш время оптический волокно становиться неотъ...,0
2,снова здравствовать большой маленький любитель...,0
3,большинство человек привыкать chromium — это...,0
4,"“ выживать сильный ,” — пафосный заявлять вс...",0


In [69]:

df = df.append(meduza, ignore_index=True)

In [70]:
df.head(10)

,text,class
0,последний шесть год пока строить команда проек...,0
1,наш время оптический волокно становиться неотъ...,0
2,снова здравствовать большой маленький любитель...,0
3,большинство человек привыкать chromium — это...,0
4,"“ выживать сильный ,” — пафосный заявлять вс...",0
5,сразу сказать — englishdom просто обожать дж...,0
6,« создавать решение решать проблема нескольк...,0
7,писать ранее приоритетный направление работа н...,0
8,речь заходить разработка современный it систем...,0
9,geektimes хабер публиковаться немало статья ра...,0


In [71]:
df.head(100)

,text,class
0,последний шесть год пока строить команда проек...,0
1,наш время оптический волокно становиться неотъ...,0
2,снова здравствовать большой маленький любитель...,0
3,большинство человек привыкать chromium — это...,0
4,"“ выживать сильный ,” — пафосный заявлять вс...",0
...,...,...
95,алексей учитель ингеборга дапкунать съемка фил...,1
96,дональд трамп позировать подписывать указ выво...,1
97,краснодарский краевой суд 11 февраль начинатьс...,1
98,70 российский адвокат девятнадцать регион рф н...,1


### Разбиваем выборку

In [72]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['class'], test_size=0.4, stratify=df['class'])

## Bag-of-Words-эмбеддинг

Понятное дело, работать с чистым текстом математические методы не умеют. Настало время получить эмбеддинги!

In [73]:
from sklearn.feature_extraction.text import CountVectorizer
bof_vect = CountVectorizer()
bof_vect.fit(np.hstack([X_train, X_test]))
bof_train = bof_vect.transform(X_train)
bof_test = bof_vect.transform(X_test)

#### P.s. обучение эмбеддинга проводится на всех данных
Посмотрим на вид нашего нового признакового пространства

In [74]:
bof_train.toarray()

array([[0, 0, 0, ..., 0, 1, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [75]:
bof_train.toarray().shape

(60, 21099)

## TF-IDF-эмбеддинг

In [76]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vect = TfidfVectorizer()
tfidf_vect.fit(np.hstack([X_train, X_test]))
tfidf_train = tfidf_vect.transform(X_train)
tfidf_test = tfidf_vect.transform(X_test)

In [77]:
tfidf_train.toarray()

array([[0.        , 0.        , 0.        , ..., 0.        , 0.01685264,
        0.        ],
       [0.01391296, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [78]:
tfidf_train.toarray().shape

(60, 21099)

## Word2vec-эмбеддинг
Поскольку w2v - это не sklearn'овский классификатор, он на выходе выдаст данные немного другого вида, и это надо будет учитывать в дальнейшей работе

In [79]:
from gensim.models import Word2Vec
X_train_w2v = X_train.apply(str.split)
X_test_w2v = X_test.apply(str.split)
w2v_vect = Word2Vec(np.hstack([X_train_w2v, X_test_w2v]), size=300, min_count=10, workers=8)

In [80]:
X_train_w2v

72    [2011, –, 2012, год, становиться, известно, ма...
5     [сразу, сказать, —, englishdom, просто, обожат...
22    [сегодняшний, время, расти, потребность, дизай...
98    [70, российский, адвокат, девятнадцать, регион...
69    [googlenобъявилnо, масштабный, обновление, сер...
29    [автор, статья, буфер, поток, двоичный, данные...
21    [вступление, помощь, расширение, мочь, создава...
66    [январь, апрель, 2015, год, количество, безраб...
82    [москва, февральский, революция, 1917, год, ка...
64    [«, наш, журналист, сидеть, тюрьма, пять, меся...
33    [публиковать, первый, часть, перевод, руководс...
86    [высокий, школа, экономика, нередкоnобвиняютnв...
37    [привет, звать, дядиченко, григорий, основател...
78    [последний, несколько, месяц, дагестан, закрыв...
88    [житель, грозный, саид, мажаев, проводить, иги...
13    [привет, gt, несколько, месяц, назад, писать, ...
65    [пионерский, линейка, 35, м, лицей, новокузнец...
40    [статья, хотеть, поделиться, размышление, 

In [81]:
w2v_vect['трамп']

<ipython-input-81-f59055bd3cb6>:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  w2v_vect['трамп']


array([-0.05663023,  0.0267729 , -0.2518072 ,  0.02021483,  0.17844947,
       -0.12992164, -0.06433745, -0.05007831,  0.14100339,  0.10468889,
       -0.06733984,  0.13827683,  0.09828866, -0.0032643 , -0.09810451,
        0.02255901, -0.13599302, -0.14709651, -0.1788689 ,  0.007878  ,
        0.03676586,  0.29272285, -0.09207076, -0.12352399, -0.06985688,
        0.06694426,  0.01146561,  0.03483351,  0.10493054,  0.05905547,
        0.20573688, -0.14005396,  0.0455571 , -0.22241738, -0.01301525,
        0.10373744, -0.06307302, -0.0541239 , -0.07645877, -0.13394845,
        0.0204288 , -0.03654367,  0.14426123,  0.04481517, -0.2807636 ,
       -0.10603178, -0.04279306,  0.25659272, -0.00270512,  0.15349193,
        0.17119285,  0.07186881,  0.04173044, -0.01643032,  0.11220619,
       -0.03719156,  0.03673344, -0.15525469,  0.17702152,  0.10999651,
        0.14337866, -0.08434441, -0.14206623, -0.18132105, -0.1472576 ,
       -0.07865372,  0.05111909,  0.08450539,  0.11186815,  0.06

In [82]:
w2v_vect.most_similar(positive="трамп")

<ipython-input-82-2f9c77173d54>:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  w2v_vect.most_similar(positive="трамп")


[('участок', 0.9999397993087769),
 ('ттп', 0.9999389052391052),
 ('условие', 0.9999358654022217),
 ('следовать', 0.9999343156814575),
 ('китайский', 0.999934196472168),
 ('сша', 0.9999333620071411),
 ('замечать', 0.9999326467514038),
 ('немного', 0.9999325275421143),
 ('официальный', 0.999932050704956),
 ('12', 0.9999310970306396)]

![title](mem/4.jpg)

### Преобразуем тексты в вектора - возьмем сумму векторов всех слов, которые входят в песню

In [84]:
import numpy as np

def text2vec(text):
    """Усредняем векторы слов"""
    vecs = []
    for word in text:
        try:
            vecs.append(w2v_vect[word])
        except KeyError:
            pass
    return np.sum(vecs, axis=0) / len(vecs)

w2v_train = X_train_w2v.apply(text2vec)
w2v_test = X_test_w2v.apply(text2vec)
w2v_train

<ipython-input-84-c817aeec8a46>:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  vecs.append(w2v_vect[word])


72    [-0.07342067, 0.029555602, -0.31997225, 0.0208...
5     [-0.07663641, 0.028607668, -0.33908632, 0.0131...
22    [-0.06582407, 0.031231662, -0.29370406, 0.0241...
98    [-0.07379555, 0.029277638, -0.32174766, 0.0210...
69    [-0.0769353, 0.030975403, -0.3323311, 0.027140...
29    [-0.06821059, 0.032395855, -0.30660748, 0.0236...
21    [-0.06717521, 0.03946428, -0.30950722, 0.02130...
66    [-0.07687815, 0.029512895, -0.32884148, 0.0252...
82    [-0.075532846, 0.029392317, -0.32421798, 0.025...
64    [-0.07697492, 0.028461708, -0.33281147, 0.0186...
33    [-0.06597941, 0.033077337, -0.29914758, 0.0256...
86    [-0.0787365, 0.029827945, -0.33759937, 0.02494...
37    [-0.06249801, 0.038426787, -0.2920328, 0.02029...
78    [-0.076828346, 0.027426064, -0.3329056, 0.0154...
88    [-0.07177265, 0.027785892, -0.3140705, 0.01638...
13    [-0.0704015, 0.03027149, -0.30831265, 0.024204...
65    [-0.07764216, 0.027274707, -0.3335789, 0.01833...
40    [-0.06436802, 0.031920843, -0.28967956, 0.

In [85]:
w2v_train.shape

(60,)

In [88]:
w2v_train[14]

array([-0.11543111, -0.11334921, -0.11346527, -0.11615738, -0.12260513,
       -0.11718865, -0.12233861, -0.11974635, -0.11869654, -0.11557027,
       -0.11831896, -0.12165917, -0.11471997, -0.11276928, -0.10983162,
       -0.11504263, -0.11410588, -0.1139378 , -0.11298995, -0.11763117,
       -0.11515383, -0.1166139 , -0.12588689, -0.1090721 , -0.11620148,
       -0.10850858, -0.11026835, -0.11352786, -0.11569104, -0.10514458,
       -0.10739814, -0.11522153, -0.112988  , -0.11728073, -0.10922539,
       -0.11611531, -0.1271126 , -0.11072429, -0.11622548, -0.1088185 ,
       -0.11108242, -0.11771754, -0.11519077, -0.11300235, -0.11706272,
       -0.11698947, -0.11598434, -0.11428291, -0.11100403, -0.11080822,
       -0.1153294 , -0.11336215, -0.11046938, -0.10787139, -0.1165375 ,
       -0.12241501, -0.11905839, -0.11032442, -0.11012229, -0.11289795],
      dtype=float32)

In [87]:
w2v_train = np.dstack(w2v_train)[0]
w2v_train.shape

(300, 60)

In [89]:
w2v_test = np.dstack(w2v_test)[0]

## Классификация текстов
Теперь у нас есть классическое признаковое описание каждого текста. Можем обучать классификаторы или придумать еще какую-то метрику.

Давайте вычислим для каждого эмбеддинга по два суммарных вектора - для текстов медузы и для текстов хабра. 
### Для Bag of Words:

In [90]:
habr_mean_bof = np.sum(bof_train[y_train == 0], axis=0)
habr_mean_bof.shape

(1, 21099)

In [91]:
meduza_mean_bof = np.sum(bof_train[y_train == 1], axis=0)

### Для TF-IDF

In [92]:
habr_mean_tfidf = np.sum(tfidf_train[y_train == 0], axis=0)
habr_mean_tfidf.shape

(1, 21099)

In [93]:
meduza_mean_tfidf = np.sum(tfidf_train[y_train == 1], axis=0)

### Для Word2Vec

In [94]:
habr_mean_w2v = np.sum(w2v_train[:, y_train == 0], axis=1)
habr_mean_w2v.shape

(300,)

In [95]:
meduza_mean_w2v = np.sum(w2v_train[:, y_train == 1], axis=1)

### Посмотрим на их вид

In [97]:
habr_mean_bof

matrix([[2, 9, 1, ..., 3, 0, 0]], dtype=int64)

In [98]:
habr_mean_tfidf

matrix([[0.02339897, 0.11521096, 0.01485279, ..., 0.03265981, 0.        ,
         0.        ]])

In [99]:
habr_mean_w2v

array([-1.96695244e+00,  9.69808996e-01, -8.84623528e+00,  6.81772768e-01,
        6.18617916e+00, -4.60914707e+00, -2.38139534e+00, -1.84322011e+00,
        5.03453541e+00,  3.65532970e+00, -2.36784506e+00,  4.69951010e+00,
        3.33283353e+00, -3.36447582e-02, -3.40008235e+00,  8.43577147e-01,
       -4.71773958e+00, -5.00152063e+00, -6.20042133e+00,  2.57354051e-01,
        1.25473750e+00,  1.00752439e+01, -3.28375411e+00, -4.38841486e+00,
       -2.48194337e+00,  2.22114778e+00,  4.33881193e-01,  1.16467559e+00,
        3.59860873e+00,  1.90778410e+00,  7.16221571e+00, -4.95813990e+00,
        1.68194556e+00, -7.53044701e+00, -4.27162468e-01,  3.66018867e+00,
       -2.16110563e+00, -1.86531198e+00, -2.58931494e+00, -4.70568657e+00,
        7.45084643e-01, -1.28393912e+00,  4.93742895e+00,  1.55386841e+00,
       -9.65359211e+00, -3.54606628e+00, -1.40921867e+00,  8.75501442e+00,
       -8.22405815e-02,  5.41082048e+00,  6.04150581e+00,  2.39184260e+00,
        1.45889139e+00, -

### А теперь построим датафреймы с результатами классификации тестовых текстов. Будем считать, что текст относится к тому издательству, с которым его косинусное расстояние больше

In [100]:
from scipy.spatial.distance import cosine
bof_habr = np.apply_along_axis(cosine, 1, bof_test.toarray(), v=habr_mean_bof)
bof_meduza = np.apply_along_axis(cosine, 1, bof_test.toarray(), v=meduza_mean_bof)

bof_results = pd.DataFrame([
    bof_habr,
    bof_meduza,
    np.maximum(bof_habr,bof_meduza) == bof_meduza,
    y_test
], index=["habr", "meduza", "predict", "class"]).T.astype(np.float)
bof_results

,habr,meduza,predict,class
0,0.639532,0.610217,0.0,0.0
1,0.440648,0.515293,1.0,0.0
2,0.523431,0.678947,1.0,0.0
3,0.705463,0.546877,0.0,1.0
4,0.548292,0.619590,1.0,0.0
5,0.697323,0.522043,0.0,1.0
6,0.442498,0.539947,1.0,0.0
7,0.712718,0.757024,1.0,0.0
8,0.735407,0.595179,0.0,1.0
9,0.731525,0.562533,0.0,1.0


In [101]:
from sklearn.metrics import accuracy_score
accuracy_score(bof_results['predict'], bof_results['class'])

0.075

In [102]:
tfidf_habr = np.apply_along_axis(cosine, 1, tfidf_test.toarray(), v=habr_mean_tfidf)
tfidf_meduza = np.apply_along_axis(cosine, 1, tfidf_test.toarray(), v=meduza_mean_tfidf)

tfidf_results = pd.DataFrame([
    tfidf_habr,
    tfidf_meduza,
    np.maximum(tfidf_habr, tfidf_meduza) == tfidf_meduza,
    y_test
], index=["habr", "meduza", "predict", "class"]).T.astype(np.float)
tfidf_results

,habr,meduza,predict,class
0,0.829445,0.808684,0.0,0.0
1,0.743598,0.794698,1.0,0.0
2,0.749985,0.884608,1.0,0.0
3,0.917039,0.796538,0.0,1.0
4,0.755556,0.796985,1.0,0.0
5,0.897898,0.751548,0.0,1.0
6,0.708172,0.780697,1.0,0.0
7,0.872157,0.916715,1.0,0.0
8,0.931054,0.853511,0.0,1.0
9,0.895825,0.737636,0.0,1.0


In [103]:
accuracy_score(tfidf_results['predict'], tfidf_results['class'])

0.075

In [104]:
w2v_habr = np.apply_along_axis(cosine, 0, w2v_test, v=habr_mean_w2v)
w2v_meduza = np.apply_along_axis(cosine, 0, w2v_test, v=meduza_mean_w2v)

w2v_results = pd.DataFrame([
    w2v_habr,
    w2v_meduza,
    np.maximum(w2v_habr, w2v_meduza) == w2v_meduza,
    y_test
], index=["habr", "meduza", "predict", "class"]).T.astype(np.float)
w2v_results

,habr,meduza,predict,class
0,0.000155,0.008885,1.0,0.0
1,0.000784,0.005697,1.0,0.0
2,0.000158,0.011887,1.0,0.0
3,0.012056,0.000110,0.0,1.0
4,0.000139,0.010835,1.0,0.0
5,0.001441,0.004062,1.0,1.0
6,0.000221,0.008416,1.0,0.0
7,0.000039,0.010643,1.0,0.0
8,0.007323,0.000244,0.0,1.0
9,0.013652,0.000426,0.0,1.0


In [105]:
accuracy_score(w2v_results['predict'], w2v_results['class'])

0.15

![title](mem/2.jpg)


### Напоследок попробуем применить поверх эмбеддингов какую-нибудь классическую модель машинного обучения, например, случайный лес

In [106]:
from sklearn.ensemble import RandomForestClassifier
RandomForestClassifier().fit(bof_train.toarray(), y_train.tolist()).score(bof_test.toarray(), y_test.tolist())

0.975

In [107]:
RandomForestClassifier().fit(tfidf_train.toarray(), y_train.tolist()).score(tfidf_test.toarray(), y_test.tolist())

0.975

In [108]:
RandomForestClassifier().fit(w2v_train.T, y_train.tolist()).score(w2v_test.T, y_test.tolist())

0.9

# Исходя из всего курса можно сделать вывод: мы готовы работать в сбере
![title](mem/1.1.jpg)